# cNMF test 
use platelet as the test sample, use conda the env **cnmf_env**

In [1]:
import os
import pandas as pd
import numpy as np
from scipy.io import mmread
import scipy.sparse as sp
import matplotlib.pyplot as plt
from IPython.display import Image
import scanpy as sc

In [2]:
if not os.path.exists('/rd2/user/xiacr/sle/output_file/cNMF/test/'):
    os.mkdir('/rd2/user/xiacr/sle/output_file/cNMF/test/')
os.chdir('/rd2/user/xiacr/sle/output_file/cNMF/test/')

In [5]:
numiter=200     # Number of NMF replicates. Set this to a larger value ~200 for real data. We set this to a relatively low value here for illustration at a faster speed
numworkers= 32   # Number of parallel factorization jobs to run. Set this to a value reflective of the number of cores on your computer.
numhvgenes=2000 # Number of over-dispersed genes to use for running the actual factorizations

## Results will be saved to [output_directory]/[run_name] which in this example is example_PBMC/cNMF/pbmc_cNMF
cnmf_path = '../../../source/cNMF/cnmf.py'

output_directory = './cNMF_out'
if not os.path.exists(output_directory):
    os.mkdir(output_directory)
run_name = 'platelet_cNMF'

## Specify the Ks to use as a space separated list in this case "5 6 7 8 9 10"
K = ' '.join([str(i) for i in range(5,11)])
## To speed this up, you can run it for only K=7-8 with the option below
#K = ' '.join([str(i) for i in range(7,9)])

seed = 14 ## Specify a seed pseudorandom number generation for reproducibility

## Path to the filtered counts dataset we output previously
countfn = './platelet_harmony_anno_sceasy.h5ad'

In [7]:
prepare_cmd = 'python %s prepare --output-dir %s --name %s -c %s -k %s --n-iter %d --total-workers %d --seed %d --numgenes %d --beta-loss frobenius' % (cnmf_path, output_directory, run_name, countfn, K, numiter, numworkers, seed, numhvgenes)
print('Prepare command assuming parallelization with %d cores:\n%s' % (numworkers, prepare_cmd))
! conda activate cnmf_env &  {prepare_cmd}

Prepare command assuming parallelization with 32 cores:
python ../../../source/cNMF/cnmf.py prepare --output-dir ./cNMF_out --name platelet_cNMF -c ./platelet_harmony_anno_sceasy.h5ad -k 5 6 7 8 9 10 --n-iter 200 --total-workers 32 --seed 14 --numgenes 2000 --beta-loss frobenius

CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.


Traceback (most recent call last):
  File "../../../source/cNMF/cnmf.py", line 7, in <module>
    import yaml
ModuleNotFoundError: No module named 'yaml'


In [9]:
## Using GNU parallel
## This took 4 minutes in our testing
worker_index = ' '.join([str(x) for x in range(numworkers)])
factorize_cmd = 'nohup parallel python %s factorize --output-dir %s --name %s --worker-index {} ::: %s' % (cnmf_path, output_directory, run_name, worker_index)
print('Factorize command to simultaneously run factorization over %d cores using GNU parallel:\n%s' % (numworkers, factorize_cmd))
#!{factorize_cmd}

Factorize command to simultaneously run factorization over 32 cores using GNU parallel:
nohup parallel python ../../../source/cNMF/cnmf.py factorize --output-dir ./cNMF_out --name platelet_cNMF --worker-index {} ::: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31
